# Webscraper für die Studentenjobs auf www.stellenwerk-hamburg.de

## Installieren und importieren benötigter Bibliotheken

Die Python-Bibliotheken *requests*, *BeautifulSoup* und *tqdm* müssen vor der ersten Verwendung installiert werden. Mit *requests* ist es möglich, über Python HTTP-Anfragen zu stellen. Mit *BeautifulSoup* kann man HTML-Code parsen und mit *tqdm* kann man sich bei langwierigen Programmausführungen Fortschrittsbalken anzeigen lassen. Die Installation geht folgendermaßen:

In [64]:
!pip install requests
!pip install bs4
!pip install tqdm

  Using cached bs4-0.0.1-py3-none-any.whl


Die ebenfalls benötigten Bibliotheken *time* (zum Verzögern der Programmausführung) und *csv* (zum Arbeiten mit csv-Dateien) sind Teil von Python und müssen nicht extra installiert werden. Sie müssen aber importiert werden, damit man sie verwenden kann.

Alle benötigten Bibliotheken importieren:

In [65]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import csv

## Anforderungen an den Web Scraper

Die **Stellenanzeigen für Studentenjobs** auf den Seiten des Stellenwerks sind [hier](https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs) zu finden.
Die Seite ist so aufgebaut, dass immer 20 Stellenangebote auf einer Seite verlinkt werden. Um die nächsten 20 Links zu Stellenangebote zu sehen, muss weitergeblättert werden. Die eigentliche Information zur Stelle findet sich hinter dem jeweiligen Link. Um an die Informationen über die Stellenangebote zu gelangen, müssen entsprechend diese Job-spezifischen Links aufgerufen werden.
Der Webscraper für das Stellenwerk muss somit
- Webseite des Stellenwerks aufrufen
- blättern und wissen, wie oft weitergeblättert werden muss (Seitenzahl)
- die 20 Links je Seite auslesen
- eine Liste aller (Seitenzahl x 20) Links zusammenstellen
- alle so gewonnen Links zu den jeweiligen Stellenanzeigen aufrufen
- die stellenbezogenen Informationen extrahieren
- die extrahierten Informationen in einer CSV-Datei abspeichern.

## Aufbau der URL des Stellenwerks

Wenn man durch die Seiten mit den Stellenanzeigen blättert, stellt man fest, dass die URL des Stellenwerks [*Query Strings*](https://de.wikipedia.org/wiki/Query-String) nutzt, um Parameter (insbesondere der Suchanfrage) an den Webserver zu senden. Die URLs haben folgenden beispielhaften Aufbau:

https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=1

Der Query-String wird mit einem Fragezeichen (?) eingeleitet und besteht aus einem oder mehreren Schlüssel=Wert-Paaren. Im Fall mehrerer Schlüssel=Wert-Paare sind diese mit einem & verbunden. Vor dem Query-String steht die Basis-URL. Somit läst sich der URL des Stellenwerks in folgende Bestandteile gliedern:

- Basis-URL: https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs
- Query-String: [keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=1](https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=1)<br>
mit folgenden Schlüssel=Wert-Paaren:
    - keywords=
    - offer_type=job_ad_b2b
    - job_category=111
    - employment_type=All
    - page=1

Diese Schlüssel=Wert-Paare des Query Strings werden im Folgenden in einem [Python-Dictionary](https://www.w3schools.com/python/python_dictionaries.asp) mit den Namen *query* gespeichert. Das Python-Dictionary wird dann - neben der Basis-URL - an *requests* übergeben, um die Webseite aufzurufen:

In [66]:
base_url = "https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs"
query = {
    'keywords': '',
    'offer_type': 'job_ad_b2b',
    'job_category': '111',
    'employment_type': 'All',
    'page': '0'
}

## Wir tarnen uns als Firefox 

Standardmäßig offenbart sich die *requests*-Bibliothek selbst, indem sie sich beim Aufrufen einer Webseite im *request header* als User-Agent nennt: **'User-Agent': 'python-requests/2.26.0'**.<br>
Um nicht sofort als Web-Scraper aufzufallen, ändern wir den entsprechenden Eintrag im *request header* und geben uns als Firefox-Browser aus (die Angaben, die Firefox sendet, habe ich mit den *Werkzeugen für Webentwickler* im Firefox-Browser ermittelt über **Netzwerkanalyse** &rightarrow; **Kopfzeilen** &rightarrow; **Anfragekopfzeilen**). Dazu werden die Angaben, die Firefox sendet, in einem Python-Dictionary namens *my_headers* gespeichert, damit sie an *requests* übergeben werden können. Damit werden die ursprünglichen Angaben zum User-Agenten überschrieben.

In [67]:
my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0'}

## Aufruf der Webseite

Nun wird ein *Get-Request* zum Aufruf der Stellenwerk-Webseite an den Webserver gestellt. Neben der Basis-URL müssen auch die Informationen aus dem Query String an *requests* übergeben werden. Dazu wird das oben erstellte Python-Dictionary mit dem Namen *query* dem funktionseigenen Argument *params* übergeben. Ebenso wird das Dictionary mit dem Namen *my_headers* dem funktionseigenen Argument *headers* übergeben, um uns so als Firefox-Browser zu tarnen.<br>
Anfrage und die Antwort des Webservers werden von *requests* in einem sog. Response-Objekt gespeichert. Dieses Response-Objekt wird dann der [Variablen](https://www.w3schools.com/python/python_variables.asp) *r* zugewiesen, so dass wir im weiteren Verlauf damit arbeiten können.

In [68]:
r = requests.get(base_url, params = query, headers = my_headers)

In [69]:
print(r.request.url) # aufgerufene URL

https://www.stellenwerk-hamburg.de/jobboerse/unternehmen/studentenjobs?keywords=&offer_type=job_ad_b2b&job_category=111&employment_type=All&page=0


In [74]:
print(r.request.headers) # an den Webserver gesendete Anfragekopfezeile

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [71]:
print(r.status_code) # War der Webseitenaufruf erfolgreich?
print(r.reason)

200
OK


## HTML-Code der Webseite speichern

Nun wird mit `.text` der übermittelte HTML-Quellcode der Stellenwerk-Webseite als Unicode-Text aus dem Response-Objekt *r* extrahiert und in der Variable *html* gespeichert:

In [72]:
html = r.text

Diese Variable wird als Input an *BeautifulSoup* übergeben und von *BeautifulSoup* in ein *BeautifulSoup*-Objekt umgewandelt. *BeautifulSoup*-Objekte erlauben es, durch den HTML-Quellcode zu navigieren, in diesem Code nach spezifischen Inhalten zu suchen und diese Inhalte zu extrahieren. Das *BeautifulSoup*-Objekt wird der Variable *bs* zugewiesen:

In [73]:
bs = BeautifulSoup(html, 'html.parser')

## Gesuchte Informationen aus dem HTML-Code auslesen

### Links zu den jeweiligen Stellenanzeigen auslesen

Aus dem Quellcode der Webseite wird das HTML-Element ausgelesen, in dem die Stellenanzeigen zu finden sind. Um dieses Element im Quellcode zu identifizieren, verwendet man die *Browserwerkzeuge* bzw. *Entwicklertools* des jeweiligen Browsers. Diese kann man in den meisten Browsern mit <kbd>Strg</kbd>+<kbd>UMSCHALTTASTE</kbd>+<kbd>I</kbd> öffnen. Hier wählt man dann den Reiter *Inspektor*.
Wenn man nun die Maus über die Webseite bewegt, ändert sich das hervorgehobene Element. Gleichzeitig wird der dazugehörge HTML-Quellcode im Fensterbereich der *Browserwerkzeuge* angezeigt. Hier sind auch rudimentäre Kenntnisse von HTML und CSS hilfreich, um die entsprechenden Elemente im Quellcode zu bestimmen.<br>
Die Informationen zu den ausgeschriebenen Jobs befinden sich in einem
[*div*](https://www.w3schools.com/tags/tag_div.ASP)-Tag, welches mit einem CSS [*class*](https://www.w3schools.com/css/css_selectors.asp)-Selektor gestaltet wurde. In diesem Fall ist *class="JobSearch-content"*. Mit diesen Informationen und der Funktion  `find()` von *BeautifulSoup* können die entsprechenden Informationen aus dem HTML-Quellcode extrahiert und in eine Variable *job_ads* gespeichert werden:

In [12]:
job_ads = bs.find('div', class_ = 'JobSearch-content')

Aus diesem so extrahierten HTML-Element werden in einem weiteren Schritt alle Links zu den Stellenanzeigen ausgelesen. Links werden in HTML über einen [*a*](https://www.w3schools.com/tags/tag_a.asp)-Tag definiert. Die Links zu den Stellenanzeigen befinden sich in einem *a*-Tag mit dem CSS-Selektor *class="Link"*. Da es hier mehrere entsprechende Elemente gibt, muss die Funktion `find_all()` von *BeautifulSoup* verwendet werden, um alle Links zu finden. Die so extrahierten Links werden in die Listen-Variable *job_links* geschrieben. Die URL, zu der der Link führt, befindet sich in dem *href*-Attribut des *a*-Tags.

In [13]:
job_links = job_ads.find_all('a', class_ = 'Link')

Anzahl an Stellenanzeigen in der Liste *job_links*:

In [14]:
print(len(job_links))

20


Es befinden sich immer 20 Links zu den Stellenanzeigen auf jeder der Unterseiten.

Als nächstes werfen wir einen Blick auf die so extrahierten Links:

In [12]:
for l in job_links:
    print(l['href'])

/jobboerse/werkstudent-innovations-wmd-hamburg-211110-502338
/jobboerse/studentische-aushilfe-im-bereich-digital-hamburg-211110-502335
/jobboerse/werkstudent-corporate-strategy-mwd-hamburg-211110-502333
/jobboerse/buerohilfe-gesucht-hamburg-211110-502330
/jobboerse/werkstudentin-operations-management-produktion-nivea-deosprays-mwd-hamburg-211110-502329
/jobboerse/wir-suchen-unterstuetzung-fuer-unseren-vertriebkundenservice-braak-211110-502328
/jobboerse/werkstudent-produktion-operations-labello-mwd-hamburg-211110-502327
/jobboerse/werkstudent-material-management-mwd-hamburg-211110-502325
/jobboerse/werkstudent-mwd-finance-und-controlling-hamburg-211026-499402
/jobboerse/werkstudent-process-development-mwd-hamburg-211110-502320
/jobboerse/werkstudent-mwd-it-erp-production-hamburg-211026-499398
/jobboerse/werkstudentin-fuer-unser-family-officevermoegenscontrolling-hamburg-hamburg-211026-499381
/jobboerse/werkstudent-procurement-marketing-services-mwd-hamburg-211110-502307
/jobboerse/mita

Wie man sieht, sind die Links unvollständig, da Protokoll (https://) und Hostname (www.stellenwerk-hamburg.de) fehlen. Die fehlenden Informationen werden den Links hinzugefügt und die nun vollständigen Links werden in der Listen-Variable *rows* gespeichert:

In [15]:
rows = []
for link in job_links:
    link = 'https://www.stellenwerk-hamburg.de' + link['href']
    rows.append(link)

Jetzt sehen die Links so aus:

In [16]:
for row in rows:
    print(row)

https://www.stellenwerk-hamburg.de/jobboerse/aushilfe-werkstudent-unserem-lifestyle-bistro-mwd-eimsbuettel-hamburg-rotherbaum-211117
https://www.stellenwerk-hamburg.de/jobboerse/paedagogische-kraefte-mw-fuer-vorschulklassen-gesucht-hamburg-211018-497612
https://www.stellenwerk-hamburg.de/jobboerse/studentische-mitarbeiter-mwdiv-im-kaufmaennischen-bereich-id-11-21-016-hamburg-211117
https://www.stellenwerk-hamburg.de/jobboerse/werkstudent-qualitaetskontrolle-wmd-hamburg-211117-503577
https://www.stellenwerk-hamburg.de/jobboerse/dozenten-mwd-gesucht-psychologie-lernstrategien-5hwoche-hamburg-211117-503576
https://www.stellenwerk-hamburg.de/jobboerse/nachhilfelehrer-mwd-hamburg-211117-503575
https://www.stellenwerk-hamburg.de/jobboerse/theater-kunst-sport-musik-alltag-hamburg-211117-503573
https://www.stellenwerk-hamburg.de/jobboerse/werkstudent-jura-mwd-hamburg-211117-503572
https://www.stellenwerk-hamburg.de/jobboerse/werkstudentin-research-analyst-mwd-hamburg-211117-503569
https://www.

# Blättern

Die Stellenanzeigen sind über mehrere Seiten à 20 Links verteilt. Wir müssen also blättern, um an alle Links zu kommen. Wie oft geblättert werden muss, kann man der Seite im unteren Bereich entnehmen.<br>
Die Information zur Anzahl der Seiten befindet sich in einem [*span*](https://www.w3schools.com/tags/tag_span.asp)-Tag mit dem CSS-Selektor *class="Pagination-text"*. Mit der Funktion  `find()` von *BeautifulSoup* kann die Informationen aus dem HTML-Quellcode extrahiert werden. Mit der Funktion `.get_text()` wird die so extrahierte Information in Text umgewandelt (andernfalls wäre sie ein *BeautifulSoup*-Objekt):

In [17]:
pages = bs.find('span', class_ = 'Pagination-text').get_text()
print(pages)
print(type(pages)) # Abfrage des Datentyps; hier: string (Zeichenkette)

Seite 1 von 55
<class 'str'>


Aus diesem String kann man die maximale Seitenzahl folgendermaßen isolieren. Der String wird über die Methode `.split()` am Leerzeichen (' ') in seine Komponenten zerlegt. Aus der Liste der Komponenten wird dann das vierte Element extrahiert. Das vierte Element hat den Index 3, da die Indexzählung bei 0 beginnt. Schließlich wird der String mit `int()` in ein Integer (ganzzahliger Wert) umgewandelt und ist somit numerisch:

In [18]:
page_max = int(pages.split(' ')[3])

In [19]:
print(page_max)
print(type(page_max)) # Abfrage des Datentyps; hier: integer (ganzzahliger Wert)

55
<class 'int'>


Nun kann diese Information genutzt werden, um zu blättern. Dazu wird eine for-Schleife in Kombination mit der Funktion `range()` verwendet. `range()` bestimmt, wie häufig die Schleife durchlaufen wird. Hier setzen wir *page_max* ein und addieren eine Eins hinzu, da `range()` standardmäßig bei Null starten und den Stoppwert nicht mit einschließen würde. In jedem Schleifendurchlauf wird im *query string* der Wert für den Schlüssel *page* (s. oben unter Aufbau der URL des Stellenwerks) um eins erhöht. Im ersten Durchlauf ist page=1, im zweiten page=2, im dritten page=3 usw. Alle Links zu den Stellenanzeigen, die auf einer Seite zu finden sind, werden dann extrahiert und zu der Liste *all_links* hinzugefügt.<br>
Nach jedem Schleifendurchlauf stoppen wir mit `time.sleep(2)` das Programm für 2 Sekunden, bevor wir die nächste Seite aufrufen. Andernfalls würden wir mit zu vielen Anfragen in kurzer Zeit eventuell den Server überlasten. Dies würde schlimmstenfalls eine [Denial-of-Service-Attacke](https://www.bsi.bund.de/DE/Themen/Verbraucherinnen-und-Verbraucher/Cyber-Sicherheitslage/Methoden-der-Cyber-Kriminalitaet/DoS-Denial-of-Service/dos-denial-of-service_node.html) darstellen.

In [63]:
all_links = [] # Anlegen einer leeren Liste zur Aufnahme der Links
for page in tqdm(range(page_max + 1)): # tqdm vor range erzeugt einen Fortschrittsbalken 
    query["page"] = page # Seitenzahl wird in jedem Schleifendurchlauf um 1 erhöht
    r = requests.get(base_url, params = query) # Webseite aufrufen
    html = r.text # HTML-Code der Webseite speichern
    bs = BeautifulSoup(html, 'html.parser') # HTML-Code an BeautifulSoup übergeben
    job_ads = bs.find('div', class_ = 'JobSearch-content') # Spalte mit den Stellenanzeigen extrahieren
    job_links = job_ads.find_all('a', class_ = 'Link') # aus dieser Spalte die Links zu den Anzeigen extrahieren
    for link in job_links: # Protokoll und Hostname jedem Link vorangestellen
        link = 'https://www.stellenwerk-hamburg.de' + link['href']
        all_links.append(link) # Links der Link-Liste hinzufügen
    time.sleep(2) # zwei Sekunden Verzögerung nach jedem Seitenaufruf: friendly scraping

  0%|          | 0/56 [00:00<?, ?it/s]

KeyboardInterrupt: 

Wie viele Elemente hat die Liste *all_links* nach dem durch alle Seiten geblättert wurde? Diese Anzahl entspricht der Anzahl an Stellenangeboten zum Scraping-Zeitpunkt.

In [21]:
print(len(all_links))

1095


Nun liegt eine Liste mit allen Links zu den Stellenangeboten vor. Jetzt muss noch bestimmt werden, welche Informationen zu den jeweiligen Stellenangeboten extrahiert werden sollen.

### Anzeige-Informationen

In [241]:
job_title = bs.find('h1', class_ = 'JobHead-title').get_text()
ad_date = bs.find('li', class_ = 'JobHead-additionalInfoItem').get_text().split(' ')[2]
ad_id = bs.select('li.JobHead-additionalInfoItem:nth-child(2)')[0].get_text()
ad_type = bs.select('li.JobHead-additionalInfoItem:nth-child(3)')[0].get_text()

Mit Hilfe einer For-Schleife kann jeder Link aufgerufen und die relevanten Informationen zu den Stellenanzeigen extrahiert werden.

In [61]:
rows = []
for link in tqdm(all_links): # tqdm vor der iterierbaren Liste 'all_links' erzeugt einen Fortschrittsbalken 
    r = requests.get(link, headers = my_headers)
    html = r.text
    bs = BeautifulSoup(html, 'html.parser')
    try:
        job_title = bs.find('h1', class_ = 'JobHead-title').get_text()
    except:
        job_title = None
    try:
        ad_date = bs.find('li', class_ = 'JobHead-additionalInfoItem').get_text().split(' ')[2]
    except:
        ad_date = None
    try:    
        ad_id = bs.select('li.JobHead-additionalInfoItem:nth-child(2)')[0].get_text()
    except:
        ad_id = None
    try:
        ad_type = bs.select('li.JobHead-additionalInfoItem:nth-child(3)')[0].get_text()
    except:
        ad_type = None
    try:
        job_type = bs.find(string = "Art der Beschäftigung").find_next('dd').get_text() # Suche nach dem Tag mit Inhalt "Art der Beschäftigung", dann das folgende dd-Tag wählen
    except:
        job_type = None
    try:
        job_duration = bs.find(string = "Zeitraum der Beschäftigung").find_next('dd').get_text()
    except:
        job_duration = None
    try:
        job_wage = bs.find(string = "Vergütung").find_next('dd').get_text()
    except:
        job_wage = None
    try:
        app_link = bs.find(string = "Bewerbungslink").find_next('dd').find_next('a', class_ = 'Link')['href']
    except:
        app_link = None
    try:
        app_email = bs.find(string = "Bewerbungs-E-Mail").find_next('a', class_ = 'IconText-mail IconText')['href']
    except:
        app_email = None
    try:
        company = bs.find(string = "Firmenname").find_next('dd').get_text()
    except:
        company = None
    try:
        loc = bs.find(string = "Standort").find_all_next('dd', limit = 2)
        location = loc[0].get_text()+ ", " + loc[1].get_text()
    except:
        loc, location = None, None
    try:
        contact = bs.find(string = "Kontaktperson").find_next('dd').get_text()
    except:
        contact = None
    try:
        job_descrip = bs.find_all('div', class_ = 'RichText')[0].get_text()
    except:
        job_descrip = None
    try:
        job_profile = bs.find_all('div', class_ = 'RichText')[1].get_text()
    except:
        job_profile = None
    row = {
    'job_title': job_title,
    'ad_date': ad_date,
    'ad_id': ad_id,
    'ad_type': ad_type,
    'job_type': job_type,
    'job_duration': job_duration,
    'job_wage': job_wage,
    'app_link': app_link,
    'app_email': app_email,
    'company': company,
    'location': location,
    'contact': contact,
    'job_descrip': job_descrip,
    'job_profile': job_profile,
    'link': link
    }
    rows.append(row)

  0%|          | 0/1095 [00:00<?, ?it/s]

## Stellen-Informationen

In [242]:
job_type = bs.find(string = "Art der Beschäftigung").find_next('dd').get_text() # Suche nach dem Tag mit Inhalt "Art der Beschäftigung", dann das folgende dd-Tag wählen
job_duration = bs.find(string = "Zeitraum der Beschäftigung").find_next('dd').get_text()
try:
    job_wage = bs.find(string = "Vergütung").find_next('dd').get_text()
except:
    job_wage = None
try:
    app_link = bs.find(string = "Bewerbungslink").find_next('dd').find_next('a', class_ = 'Link')['href']
except:
    app_link = None
try:
    app_email = bs.find(string = "Bewerbungs-E-Mail").find_next('a', class_ = 'IconText-mail IconText')['href']
except:
    app_email = None

## Firmen-Informationen

In [243]:
company = bs.find(string = "Firmenname").find_next('dd').get_text()
loc = bs.find(string = "Standort").find_all_next('dd', limit = 2)
location = loc[0].get_text()+ ", " + loc[1].get_text()
contact = bs.find(string = "Kontaktperson").find_next('dd').get_text()

## Beschreibung der Stelle

In [244]:
job_descrip = bs.find_all('div', class_ = 'RichText')[0].get_text()

## Anforderungsprofil der Stelle

In [245]:
job_profile = bs.find_all('div', class_ = 'RichText')[1].get_text()

## Extrahierte Daten in eine CSV-Datei schreiben

In [62]:
file = 'daten/job_ads.csv'
with open(file, mode = 'w', encoding = 'utf-8', newline = '') as f:
    col_names = ['link', 'job_title', 'ad_date', 'ad_id', 'ad_type', 'job_type', 'job_duration', 'job_wage', 'app_link', 'app_email', 'company', 'location', 'contact', 'job_descrip', 'job_profile', 'link']
    writer = csv.DictWriter(f, fieldnames = col_names)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)